
This doc is trying to find out why 768 is used and important in nlp transformers

In [8]:

def is_prime(n):
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


def sqrt(n: int):
    return int(n**0.5) + 1


def factorise(n: int):
    factors = []
    i = 2
    while n > 1:
        for i in range(2, sqrt(n)):
            if n % i == 0:
                factors.append(i)
                n = n // i
                print(n, i)
                break

        if is_prime(n):
            factors.append(n)
            break

    f = set(factors)

    return factors , { i: factors.count(i) for i in f }



f, exponents = factorise(768)
print( ' * '.join(f'{i}^{exp}' for i, exp in exponents.items()))


384 2
192 2
96 2
48 2
24 2
12 2
6 2
3 2
2^8 * 3^1



## BERT_base

### Layers
Sequential([
12 Encoders blocks
Linear layer only for the first token [CLS] (1,hidden)
Linear (hidden, vocab_size)
Softmax layer
])
Encoder block:
Sequential([
    MultiHeadAttention( 12 heads, 768 dim)
    Add & Norm
    ProjectionLayer( 768 , 3072 )
    Activation
    ProjectionLayer( 3072 , 768 )
    Add & Norm
])
## Explaination

The token are prepended with a special token [CLS] and appended with a special token [SEP]
The token are then passed through the embedding layer and the positional encoding layer
The output of the positional encoding layer is passed through the 12 encoder blocks
So the batch shape is (batch_size, seq_len, 768)
The output of the last encoder block is passed through a linear layer to get the logits
Note that 768 is the hidden size of the model and the size of the embedding token
## Training
The output of the last encoder block is clipped to the first token [CLS]
Vocab size is the number of classes
The encoder block are trained in pretraining, the linear layer is trained in fine-tuning by prepending the sequences with a  [CLS] and training the model to predict the class of the sequence
